## Analyze MRIQC IQMs


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
import json
import ptitprince as pt


### Load data (paths are specified in a local config file)

In [ ]:
local_config_f = Path('../../local_config.json')
if local_config_f.exists():
    with open(local_config_f) as f:
        local_config = json.load(f)
else:
    print(f'Specify a local_config.json with path to nipoppy DATASET_DIR')

print('local_config:', local_config)


dx_color_palette = local_config['plot_styles']['DX_COLOR_PALETTE']
palette = [dx_color_palette["PD"], dx_color_palette["control"]]

sns.palplot(palette)

In [ ]:
dataset_dir = local_config['DATASET_DIR']
current_release = local_config['DATASET_RELEASE']

pipeline = "mriqc"
pipeline_version = "23.1.0"
session = "ses-01"

# Current nipoppy manifest
manifest_csv = f"{dataset_dir}/manifest.csv"

# tabular data
tabular_dir = f"{dataset_dir}/tabular/"

# demographics
demographics_csv = f"{tabular_dir}/demographics.csv"

# Dx
dx_csv = f"{tabular_dir}/assessments/diagnosis.csv"

# derivative data
derivatives_dir = f"{dataset_dir}/derivatives/"

# IDPs
idp_dir = f"{derivatives_dir}/{pipeline}/{pipeline_version}/idp/"

anat_csv = f"{idp_dir}/{session}/anat_IQM.csv"
func_csv = f"{idp_dir}/{session}/func_IQM.csv"

# figures
figs_dir = f"{dataset_dir}/results/{session}/mriqc/figs/"

### manifest

In [ ]:
manifest_cols = ["participant_id", "visit", "session"]
nipoppy_df = pd.read_csv(manifest_csv)
nipoppy_df = nipoppy_df[manifest_cols] 
nipoppy_participants = nipoppy_df["participant_id"].unique()
n_nipoppy_participants = len(nipoppy_participants)
print(f"nipoppy participants: {n_nipoppy_participants}")
nipoppy_df.head()

### Diagnosis info
- as confirmed later by the clinicians

In [ ]:
dx_df = pd.read_csv(dx_csv)
dx_df = dx_df[dx_df["redcap_event_name"] == "Baseline (Arm 1: C-OPN)"]

control_participants = dx_df[dx_df["diagnosis_group_for_analysis"] == "control"]["participant_id"].unique()
PD_participants = dx_df[dx_df["diagnosis_group_for_analysis"] == "PD"]["participant_id"].unique()

all_participants = list(control_participants) + list(PD_participants)

dx_df = dx_df[dx_df["redcap_event_name"] == "Baseline (Arm 1: C-OPN)"][["participant_id", "diagnosis_group_for_analysis"]]

print(f"PD + control: {len(all_participants)}")
print(f"Control: {len(control_participants)}")
print(f"PD: {len(PD_participants)}")

dx_df.head()

### Read mriqc output

In [ ]:
anat_df = pd.read_csv(anat_csv)
func_df = pd.read_csv(func_csv)

func_df.head()

### Plot anat IQMs

In [ ]:
save_fig = True

IQM_cols = ["cnr", "snr_gm", "snr_wm", "snr_csf", "cjv", "efc", "fber", "rpve_gm", "rpve_wm", "rpve_csf"] #

demo_cols = ["participant_id", "session_id", "diagnosis_group_for_analysis"]
plot_groups = ["control","PD"]

anat_dx_df = pd.merge(anat_df, dx_df, on="participant_id")[IQM_cols + demo_cols]
anat_dx_df = anat_dx_df[anat_dx_df["diagnosis_group_for_analysis"].isin(plot_groups)]

anat_df_melt = anat_dx_df.melt(
    id_vars=demo_cols, 
    var_name="IQM", 
    value_name="value")

plot_df = anat_df_melt.copy()
plot_df["group"] = plot_df["diagnosis_group_for_analysis"] # rename for plotting

n_participants = plot_df["participant_id"].nunique()
print(f"n_participants: {n_participants}")

participants_per_group = plot_df.groupby(["group"])["participant_id"].nunique()
print(f"participants_per_group: {participants_per_group}")


sns.set_theme(font_scale=3)
with sns.axes_style("whitegrid"):
    # g = sns.catplot(x="group",y="value", col="IQM", col_wrap=5, sharey=False, #boxprops={'alpha': 0.4},
    #                 kind="violin", inner="stick", 
    #                 palette=palette, data=plot_df, aspect=1, height=8)
    
    # Rainclouds with FacetGrid
    g = sns.FacetGrid(plot_df, col = "IQM", col_wrap=5,height = 6, sharex=True, sharey=False)
    g = g.map_dataframe(pt.RainCloud, x = "group", y = "value", data = plot_df, palette=palette, #bw = 0.2, 
                        width_viol = 0.5, ax = None, 
                        orient = "v", box_showfliers = False,
                        point_size=10, point_alpha=0.3, 
                        box_linewidth = 5, cloud_alpha = 0.9, dodge = True)
    
    g.set_titles(row_template = '{row_name}', col_template = '{col_name}')
    g.set_xlabels("")
    g.set_ylabels("")
    g.despine(left=True)

if save_fig:
    print(f"Saving figure to {figs_dir}")
    g.savefig(f"{figs_dir}/QC_anat.png")


### Plot func IQMs

In [ ]:
save_fig = True

IQM_cols = ["snr", "efc", "fber", "tsnr", "dvars_nstd", "gcor", "fd_mean", "gsr_x", "gsr_y", "aqi"] #"fwhm_avg", "aor",

demo_cols = ["participant_id", "session_id", "diagnosis_group_for_analysis"]
plot_groups = ["control","PD"]

func_dx_df = pd.merge(func_df, dx_df, on="participant_id")[IQM_cols + demo_cols]
func_dx_df = func_dx_df[func_dx_df["diagnosis_group_for_analysis"].isin(plot_groups)]

func_df_melt = func_dx_df.melt(
    id_vars=demo_cols, 
    var_name="IQM", 
    value_name="value")

plot_df = func_df_melt.copy()
plot_df["group"] = plot_df["diagnosis_group_for_analysis"] # rename for plotting

n_participants = plot_df["participant_id"].nunique()
print(f"n_participants: {n_participants}")

participants_per_group = plot_df.groupby(["group"])["participant_id"].nunique()
print(f"participants_per_group: {participants_per_group}")


sns.set_theme(font_scale=3)
with sns.axes_style("whitegrid"):
    # g = sns.catplot(x="group",y="value", col="IQM", col_wrap=5, sharey=False,
    #                 kind="box",palette=palette, data=plot_df, aspect=1, height=8)
    # # g.tick_params(axis='x', rotation=90, labelsize=14)

    # Rainclouds with FacetGrid
    g = sns.FacetGrid(plot_df, col = "IQM", col_wrap=5, height = 6, sharex=True, sharey=False)
    g = g.map_dataframe(pt.RainCloud, x = "group", y = "value", data = plot_df, palette=palette, #bw = 0.2, 
                        width_viol =  0.5, ax = None, 
                        orient = "v", box_showfliers = False,
                        point_size=10, point_alpha=0.3, 
                        box_linewidth = 5, cloud_alpha = 0.9, dodge = True)
    
    g.set_titles(row_template = '{row_name}', col_template = '{col_name}')
    g.set_xlabels("")
    g.set_ylabels("")
    g.despine(left=True)

if save_fig:
    print(f"Saving figure to {figs_dir}")
    g.savefig(f"{figs_dir}/QC_func.png")

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
corr_df = func_df[IQM_cols].corr()
n_color_bins = 10
max_color = '#e63946'

cmap = LinearSegmentedColormap.from_list('', ['white', max_color], n_color_bins)
with sns.axes_style("white"):
    sns.set_context("paper")
    fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(15, 7))
    cbar_ax = fig.add_axes([.905, .2, .01, .6])
    
    g = sns.heatmap(corr_df, square=False, xticklabels=True, yticklabels=True, cmap=cmap, ax=ax, 
                        cbar_kws= {'label': 'conmat-count'},cbar_ax=cbar_ax)